In [1]:
!pwd

/Users/irubachev/Documents/CS/cvision/03-sign-classification


In [2]:
%matplotlib inline
import numpy as np

from skimage.io import imread
from skimage.feature import hog
from skimage.color import rgb2gray
from skimage.transform import resize

from matplotlib.pyplot import imshow

from os.path import join

from sklearn.svm import SVC as svm_cl
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split, cross_val_score

from ipywidgets import FloatProgress
from IPython.display import display

from time import time

import warnings
warnings.simplefilter('ignore')

In [3]:
def preprocess_img(img, size=(64, 64), preserve_rgb=False):
    if not preserve_rgb:
        img = rgb2gray(img)
    
    img = resize(img, size)
    return img

In [4]:
def hog_histograms(magnitude, angles, cellx, celly, n_cellx, n_celly,
                   orientations, orientation_histogram):
    angle_period = 180. / orientations
    for cy_cnt in range(n_celly):
        for cx_cnt in range(n_cellx):
            pnt_x = cellx * cx_cnt
            pnt_y = celly * cy_cnt
            
            histogram = np.zeros(orientations)
            for y in range(pnt_y, pnt_y + celly):
                for x in range(pnt_x, pnt_x + cellx):
                    _angle = angles[y][x]
                    bin_n = int(_angle // angle_period)
                    relative_angle = _angle % angle_period
                    partition = relative_angle / angle_period
                    histogram[bin_n % orientations] += (1. - partition) * magnitude[y][x]
                    histogram[(bin_n + 1) % orientations] += partition * magnitude[y][x]
            
            orientation_histogram[cy_cnt][cx_cnt] = histogram
    
    return orientation_histogram

In [5]:
def extract_hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), 
                norm_eps=1e-12, apply_sqrt=True):
    img = preprocess_img(img)
    
    if apply_sqrt:
        img = np.sqrt(img)

    dy, dx = np.gradient(img, .5)
    
    magn = np.hypot(dx, dy)
    angl = np.rad2deg(np.arctan2(dy, dx)) % 180
        
    sz_y, sz_x = img.shape
    cellx, celly = pixels_per_cell
    blockx, blocky = cells_per_block
    
    n_cellx = int(sz_x // cellx)
    n_celly = int(sz_y // celly)
    
    orientation_histogram = np.zeros((n_celly, n_cellx, orientations))
    orientation_histogram = hog_histograms(magn, angl, cellx, celly, n_cellx, n_celly,
                                           orientations, orientation_histogram)
    
    n_blockx = n_cellx - blockx + 1
    n_blocky = n_celly - blocky + 1
    
    norm_blocks = np.zeros((n_blocky, n_blockx, blocky, blockx, orientations))
    
    for y in range(n_blocky):
        for x in range(n_blockx):
            block = orientation_histogram[y: y + blocky, x: x + blockx, :]
            norm_blocks[y, x, :] = block / np.sqrt(np.sum(block ** 2) + norm_eps ** 2)
    
    return np.ravel(norm_blocks)

In [6]:
def preprocess_data(data_dir, filename):
    fgt = open(join(data_dir, filename))
    next(fgt)
    lines = fgt.readlines()

    filenames = []
    labels = np.zeros(len(lines))
    for i, line in enumerate(lines):
        filename, label = line.rstrip('\n').split(',')
        filenames.append(filename)
        labels[i] = int(label)

    return filenames, labels

In [7]:
def extract_features(path, filenames):
    hog_length = len(extract_hog(imread(join(path, filenames[0]), plugin='matplotlib')))
    data = np.zeros((len(filenames), hog_length))
    
    f = FloatProgress(min=0, max=len(filenames)) # instantiate the bar
    display(f)
    
    for i in range(0, len(filenames)):
        f.value += 1
        
        filename = join(path, filenames[i])
        data[i, :] = extract_hog(imread(filename, plugin='matplotlib'))
    return data

In [11]:
def fit_and_classify(train_features, train_labels, test_features, return_model=False):
    model = svm_cl(kernel='linear', C=100.0)
    model.fit(train_features, train_labels)
    prdct = model.predict(test_features)
    
    if return_model:
        return prdct, model
    return prdct

In [10]:
%%time
data_path = "/Users/irubachev/Documents/CS/cvision/03-sign-classification/data/train"
data_filename = "gt.csv"

filenames, labels = preprocess_data(data_path, data_filename)

data = extract_features(data_path, filenames)

CPU times: user 9min 34s, sys: 20.5 s, total: 9min 54s
Wall time: 10min 3s


In [12]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.4, random_state=42)

In [13]:
y_predicted, model = fit_and_classify(X_train, y_train, X_test, return_model=True)

In [14]:
print(model.score(X_test, y_test))

0.97908696761


In [22]:
model = svm_cl(kernel='rbf', C=100.0)
scores = cross_val_score(model, data, labels, cv=5, verbose=1, n_jobs=-1)

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 71.9min finished


In [23]:
np.mean(scores)

0.93175012321691641

In [24]:
scores

array([ 0.93088498,  0.93649579,  0.93496557,  0.93407294,  0.92233134])